# Building an Augmented LLM Agent

This notebook demonstrates the evolution of a Large Language Model (LLM) agent from a simple chatbot to a sophisticated system with memory, retrieval, and tool use capabilities. We'll follow the approach outlined in Anthropic's "Building Effective Agents" article, implementing each capability step by step.

Key concepts we'll cover:
1. Basic LLM interactions
2. Adding conversation memory
3. Implementing RAG (Retrieval Augmented Generation)
4. Integrating specialized tools

Each section builds upon the previous one, demonstrating how we can progressively enhance our LLM's capabilities.

In [1]:
# Сначала импортируем все необходимые библиотеки:
# - os: для работы с переменными окружения, где мы храним наши ключи
# - getpass: позволяет безопасно вводить секретные данные (пароли, ключи API)
# - typing: добавляет подсказки о типах данных, делая код более понятным
# - json: для работы с форматом JSON, который используется в API
# - requests: популярная библиотека для отправки HTTP запросов
# - pydantic: для валидации данных и создания моделей

import os
import getpass
from typing import List, Dict, Optional
import json
import requests
from pydantic import BaseModel, Field

In [2]:
# Создаем функцию для безопасной установки ключей API
# Это важно, потому что мы не хотим хранить ключи в коде

def _set_env(var: str):
    # Проверяем, есть ли уже ключ в переменных окружения
    if not os.environ.get(var):
        # Если нет - запрашиваем его у пользователя безопасным способом
        os.environ[var] = getpass.getpass(f"{var}: ")

# Запрашиваем необходимые ключи API:
# 1. OPENAI_API_KEY - для доступа к GPT моделям и эмбеддингам
# 2. PINECONE_API_KEY - для доступа к векторной базе данных
_set_env("OPENAI_API_KEY")
_set_env("PINECONE_API_KEY")

# [VIDEO] Задаем имя индекса в Pinecone для хранения наших векторов
# Pinecone - это векторная база данных, где мы храним эмбеддинги текста
index_name = "augmentedllm"

OPENAI_API_KEY:  ········
PINECONE_API_KEY:  ········


In [3]:
# Создаем класс для тестирования диалогов с агентом
# Этот класс помогает более удобочитаемо отображать переписку между пользователем и агентом

# Helper class for running dialogue tests
class DialogueTest:
    """Simple framework for testing LLM dialogue capabilities."""
    
    def __init__(self, agent):
        """Initialize with an LLM agent to test."""
        # agent - это наш бот (LLM), который будет отвечать на сообщения
        # Мы сохраняем его в self.agent, чтобы использовать позже
        self.agent = agent
    
    def chat(self, message: str) -> str:
        """Send a message and get the response."""
        # Метод chat:
        # 1. Принимает сообщение от пользователя
        # 2. Удобно выводит это сообщение
        print(f"\nUser: {message}")
        # 3. Получает ответ от агента через process_message
        response = self.agent.process_message(message)
        # 4. Выводит ответ
        print(f"Assistant: {response}")
        # 5. Возвращает ответ для дальнейшего использования
        return response

## Part 1: Basic LLM Implementation

In this first step, we create a simple chatbot that can interact with OpenAI's API. This basic implementation:
- Makes individual API calls for each message
- Has no memory of previous interactions
- Uses a simple system prompt

Key Points:
- Each interaction is independent
- No context preservation between messages
- Demonstrates bare minimum for LLM interaction

Let's see how this basic version works and why we need more sophisticated features.

In [5]:
# Создаем базовый класс для работы с LLM (OpenAI)
# Это самая простая версия бота, которая просто отправляет запросы к OpenAI
class BasicLLM:
    """Basic LLM with just chat capability."""
    
    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        # При создании бота указываем:
        # - api_key: ключ для доступа к API OpenAI
        # - model: какую модель использовать (по умолчанию gpt-4o-mini)
        self.api_key = api_key
        self.model = model
        
    def process_message(self, message: str) -> str:
        """Process a single message."""
        # Главный метод для обработки сообщений пользователя:
        try:
            # Создаем базовый запрос к OpenAI API
            response = requests.post(
                # URL API OpenAI для чат-моделей
                "https://api.openai.com/v1/chat/completions",
                # Заголовки:
                # - Authorization: указываем наш ключ API
                # - Content-Type: говорим, что отправляем JSON
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model, # Какую модель использовать
                    # Формируем тело запроса:
                    # - system content Системное сообщение - задает роль боту
                    # - user content содержит сообщение пользователя
                    "messages": [
                        {
                            "role": "system",
                            "content": "You are a helpful AI assistant."
                        },
                        {
                            "role": "user",
                            "content": message
                        }
                    ]
                }
            )
            # Проверяем успешность запроса
            response.raise_for_status()
            # Извлекаем текст ответа из JSON:
            # 1. response.json() - преобразуем ответ в словарь
            # 2. ['choices'][0] - берем первый (и единственный) вариант ответа
            # 3. ['message']['content'] - получаем текст ответа
            return response.json()['choices'][0]['message']['content']
            
        except Exception as e:
            # Если что-то пошло не так:
            # - Логируем ошибку
            # - Возвращаем понятное сообщение
            return f"Error: {str(e)}"

In [7]:
# Test Basic LLM
basic_llm = BasicLLM(os.environ["OPENAI_API_KEY"])
basic_test = DialogueTest(basic_llm)

print("\nTesting Basic LLM:")
basic_test.chat("Hi! My name is Viacheslav.")
basic_test.chat("Do you remember my name?")


Testing Basic LLM:

User: Hi! My name is Alice.
Assistant: Hi Alice! How can I assist you today?

User: Do you remember my name?
Assistant: I don’t have the ability to remember personal details or past interactions, including names. How can I assist you today?


'I don’t have the ability to remember personal details or past interactions, including names. How can I assist you today?'

## Part 2: Adding Conversation Memory

Now we enhance our LLM with conversation memory. This allows the agent to:
- Remember previous interactions
- Maintain context throughout a conversation
- Provide more coherent and contextual responses

Key Improvements:
- Stores conversation history
- Includes past messages in new requests
- Uses Pydantic for structured data management

This step significantly improves the agent's ability to maintain meaningful conversations.

In [11]:
# Создаем класс Message с помощью Pydantic
# Pydantic - это библиотека, которая помогает нам:
# 1. Проверять правильность данных
# 2. Автоматически преобразовывать данные в нужный формат
# 3. Делать код более надежным
class Message(BaseModel):
    """Single message in conversation history."""
    # У каждого сообщения есть:
    # - role: кто отправил (system/user/assistant)
    # - content: текст сообщения
    role: str
    content: str

# Создаем улучшенную версию Агента с памятью.
# Для этого наследуемся от BasicLLM (получаем всю его функциональность)
# и добавляем новые возможности
class MemoryLLM(BasicLLM):
    """LLM with conversation memory."""
    
    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        # При создании:
        # 1. Вызываем инициализацию родительского класса (super)
        # 2. Создаем пустой список для хранения истории
        super().__init__(api_key, model)
        # history будет хранить объекты типа Message
        # List[Message] означает "список объектов класса Message"
        self.history: List[Message] = []
        
    def process_message(self, message: str) -> str:
        """Process message with conversation history."""
        try:
            # Prepare messages including history
            # Готовим список сообщений для API:
            # 1. Системное сообщение (задаем роль ассистента)
            messages = [{
                "role": "system", 
                "content": "You are a helpful marketing assistant. Answer just if you sure about correct info 100%. If not say you not sure."
            }]

            # Добавляем историю разговора:
            # 1. Берем все сообщения из self.history
            # 2. Преобразуем каждое в словарь через msg.dict()
            # 3. Добавляем их в список сообщений
            messages.extend([msg.dict() for msg in self.history])
            # Добавляем текущее сообщение пользователя
            messages.append({"role": "user", "content": message})

            # Отправляем запрос к API:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model,
                    "messages": messages # В отличие от BasicLLM, теперь мы отправляем всю историю разговора
                }
            )
            response.raise_for_status()
            
            # Get response and update history
            # Получаем ответ и обновляем историю:
            # 1. Извлекаем текст ответа из JSON
            assistant_message = response.json()['choices'][0]['message']['content']
            # Сохраняем в историю:
            # 1. Сообщение пользователя
            self.history.append(Message(role="user", content=message))
            # 2. Ответ ассистента
            self.history.append(Message(role="assistant", content=assistant_message))
            
            return assistant_message
            
        except Exception as e:
            return f"Error: {str(e)}"

In [12]:
 # Test 2: LLM with Memory
print("\n=== Testing LLM with Memory ===")
memory_llm = MemoryLLM(os.environ["OPENAI_API_KEY"])
memory_test = DialogueTest(memory_llm)
memory_test.chat("Hi! My name is Viacheslav.")
memory_test.chat("Do you remember my name?") 
memory_test.chat("What you khow about The Global Sourcing RFI Company?") 


=== Testing LLM with Memory ===

User: Hi! My name is Viacheslav.
Assistant: Hello Viacheslav! How can I assist you today?

User: Do you remember my name?
Assistant: Yes, your name is Viacheslav. How can I help you today?

User: What you khow about The Global Sourcing RFI Company?
Assistant: I'm not sure about specific current details regarding The Global Sourcing RFI Company, as my training data only goes up to October 2023, and I don't have direct knowledge about every company. If you need information about their services, reputation, or other specific inquiries, I recommend checking their official website or recent news articles for the most accurate and up-to-date information.


"I'm not sure about specific current details regarding The Global Sourcing RFI Company, as my training data only goes up to October 2023, and I don't have direct knowledge about every company. If you need information about their services, reputation, or other specific inquiries, I recommend checking their official website or recent news articles for the most accurate and up-to-date information."

## Part 3a: Implementing RAG - Document Processing

RAG (Retrieval Augmented Generation) enhances our LLM with the ability to use external knowledge. This section focuses on processing and storing documents:

Key Components:
1. Document Chunking: Breaking documents into manageable pieces
2. Embedding Generation: Converting text to vector representations
3. Vector Storage: Storing embeddings in Pinecone for efficient retrieval

This step sets up the infrastructure for knowledge-enhanced responses.

In [20]:
# DocumentProcessor - это класс для разбивки общего текста на маленькие части (чанки)
# Это нужно потому что:
# 1. Большие тексты не помещаются в контекст LLM целиком
# 2. Маленькие части легче искать и сравнивать
# 3. Можно находить только релевантные части текста
class DocumentProcessor:
    """Processes documents into chunks."""
    
    def __init__(self, 
                 chunk_size: int = 1000, 
                 chunk_overlap: int = 200):
        # При создании указываем:
        # - chunk_size: сколько символов будет в одном кусочке
        # - chunk_overlap: сколько символов будет пересекаться между кусочками
        # Пересечение нужно, чтобы не разрывать предложения на середине
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def create_chunks(self, text: str) -> List[Dict[str, str]]:
        """Split text into overlapping chunks."""
        # Сначала проверяем входные данные:
        # 1. Текст не должен быть пустым
        if not text:
            raise ValueError("Empty text provided")
        # 2. Размер чанка должен быть больше перекрытия
        if self.chunk_size <= self.chunk_overlap:
            raise ValueError("chunk_size must be greater than chunk_overlap")
        
        chunks = []
        start = 0
        # Алгоритм разбивки:
        # 1. Берем кусок (chunk) текста размером chunk_size
        # 2. Добавляем его (chunks.append) в список вместе с метаданными
        # 3. Сдвигаемся вперед на (chunk_size - chunk_overlap)
        # Повторяем пока не обработаем весь текст
        while start < len(text):
            end = start + self.chunk_size
            chunk = text[start:end]     
            chunks.append({
                # Каждый чанк - это словарь с:
                # - text: сам текст
                # - metadata: дополнительная информация (где начинается и заканчивается)
                'text': chunk,
                'metadata': {'start_char': start, 'end_char': end}
            })
            
            start += self.chunk_size - self.chunk_overlap

        # Выводим информацию для отладки
        print(f"\nCreated {len(chunks)} chunks")
        print(f"Sample chunk (first 50 chars): {chunks[0]['text'][:50]}...")
        return chunks

In [21]:
# EmbeddingGenerator - класс для создания векторных представлений текста
# Эмбеддинги - это способ представить текст в виде чисел (векторов)
# Это нужно, чтобы:
# 1. Можно было сравнивать тексты математически
# 2. Находить похожие тексты быстро
# 3. Хранить тексты в векторной базе данных
class EmbeddingGenerator:
    """Handles text embedding generation."""
    
    def __init__(self, api_key: str):
        # Настраиваем доступ к API OpenAI:
        # 1. Сохраняем ключ API
        # 2. Задаем URL для создания эмбеддингов
        self.api_key = api_key
        self.api_url = "https://api.openai.com/v1/embeddings"
        
    def create_embedding(self, text: str) -> List[float]:
        """Generate embedding for text."""
        # Проверяем, что текст не пустой
        if not text.strip():
            raise ValueError("Empty text provided")

        # [Отправляем запрос к API OpenAI:
        # 1. URL для создания эмбеддингов
        # 2. Заголовки с ключом API
        # 3. json Данные: модель и текст
        response = requests.post(
            self.api_url,
            headers={
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            },
            json={
                # Используем модель text-embedding-3-small
                # Она создает вектора размером 1536 (!)
                "model": "text-embedding-3-small",
                "input": text
            }
        )
        response.raise_for_status()
        
        # Получаем вектор из ответа:
        # 1. Преобразуем ответ в JSON
        # 2. Берем первый (и единственный) эмбеддинг
        embedding = response.json()['data'][0]['embedding']
        # Выводим первые числа для отладки
        print(f"Generated embedding, sample of first 3 dimensions: {embedding[:3]}")
        return embedding

In [22]:
# VectorStore - класс для работы с векторной базой данных Pinecone
# Pinecone помогает:
# 1. Хранить эмбеддинги (вектора)
# 2. Искать релевантные векторы
# 3. Организовывать данные (namespace)
class VectorStore:
    """Manages vector storage operations."""

    def __init__(self, api_key: str, index_name: str):
        # Настраиваем подключение к Pinecone:
        # Сохраняем ключ API и имя индекса
        self.api_key = api_key
        self.index_name = index_name
        # Задаем заголовки для запросов
        self.headers = {
            "Api-Key": self.api_key,
            "Content-Type": "application/json",
            "X-Pinecone-API-Version": "2024-07"
        }
        # Создаем кэш для хостов (чтобы не запрашивать каждый раз)
        self.index_host_cache = {}
        
    def describe_index(self) -> str:
        """Get or retrieve cached index host."""
        # Сначала проверяем кэш:
        if self.index_name in self.index_host_cache:
            return self.index_host_cache[self.index_name]

        # Если в кэше нет, делаем запрос к API Pinecone:
        # Формируем URL с именем индекса
        url = f"https://api.pinecone.io/indexes/{self.index_name}"
        # Отправляем GET запрос с заголовками
        response = requests.get(url, headers=self.headers)
        response.raise_for_status()

        # Получаем хост из ответа:
        # Извлекаем хост из JSON
        host = response.json()["host"]
        # Сохраняем в кэш и возвращаем хост
        self.index_host_cache[self.index_name] = host
        print(f"\nConnected to Pinecone host: {host}")
        return host
        
    def query_vectors(self, 
                      query_vector: List[float], 
                      top_k: int = 3, 
                      namespace: str = "") -> List[Dict]:
        """Query for most similar vectors."""
        # Сначала получаем хост индекса
        host = self.describe_index()

        # Формируем запрос на поиск:
        url = f"https://{host}/query"
        data = {
            "vector": query_vector, # Вектор, который ищем 
            "topK": top_k, # Сколько похожих векторов вернуть
            "namespace": namespace, # В какой коллекции искать
            "includeMetadata": True # Включать ли метаданные в ответ
        }

        # Делаем запрос к API:
        print(f"\nQuerying Pinecone for top {top_k} matches")
        response = requests.post(url, headers=self.headers, json=data)
        response.raise_for_status()
        
        # Возвращаем найденные совпадения
        matches = response.json()['matches']
        return matches


    def store_vectors(self, vectors: List[Dict], namespace: str = "") -> Dict:
        """Store vectors in Pinecone."""
        # Получаем хост для индекса
        host = self.describe_index()
        # Формируем URL для сохранения векторов
        # 'upsert' означает "обновить если существует, создать если нет"
        url = f"https://{host}/vectors/upsert"
        # Готовим данные для отправки:    
        data = {
            "vectors": vectors, # - vectors: список векторов с их ID и метаданными
            "namespace": namespace # - namespace: коллекция, где будем хранить (как папка)
        }
        
        # Выводим информацию о загрузке:
        print(f"\nUploading {len(vectors)} vectors to Pinecone") # Сколько векторов загружаем
        print(f"Sample vector metadata: {vectors[0]['metadata']}") # Пример метаданных первого вектора

        # Отправляем запрос и получаем результат
        response = requests.post(url, headers=self.headers, json=data)    
        response.raise_for_status()
        result = response.json()
        print(f"Pinecone response: {result}")
        return result

In [23]:
# Функция для полной обработки документа:
# 1. Разбивает на части
# 2. Создает эмбеддинги
# 3. Сохраняет в базу данных
def process_and_store_document(text: str, 
                               document_id: str, 
                               openai_key: str, 
                               pinecone_key: str, 
                               index_name: str) -> bool:
    """Process document and store in vector database."""
    try:
        # Initialize components
        # Создаем все необходимые компоненты:
        # 1. DocumentProcessor - для разбивки текста
        # 2. EmbeddingGenerator - для создания эмбеддингов
        # 3. VectorStore - для сохранения в базу
        processor = DocumentProcessor()
        embedding_gen = EmbeddingGenerator(openai_key)
        vector_store = VectorStore(pinecone_key, index_name)
        
        # Create chunks
        # Разбиваем текст на части
        chunks = processor.create_chunks(text)
        
        # Generate embeddings and prepare vectors
        # Для каждой части:
        vectors = []
        
        for i, chunk in enumerate(chunks):
            embedding = embedding_gen.create_embedding(chunk['text']) # Создаем эмбеддинг (вектор)
            if embedding:
                 # Формируем структуру для сохранения и Добавляем в список vectors
                vectors.append({
                    'id': f"{document_id}-{i}", # Уникальный ID для каждой части
                    'values': embedding, # Сам вектор
                    'metadata': { # Дополнительная информация
                        'text': chunk['text'], # Исходный текст
                        'document_id': document_id, # ID документа
                        **chunk['metadata'] # Другие метаданные
                    }
                })
        
        # Store vectors
        # Сохраняем вектора если они есть
        if vectors:
            vector_store.store_vectors(vectors)
            return True
        return False
            
    except Exception as e:
        print(f"Error processing document: {e}")
        return False

## Part 3b: RAG-Enabled LLM

Now we integrate the RAG capabilities with our LLM. This combination allows the agent to:
- Search for relevant information in our document base
- Include retrieved context in its responses
- Provide more informed and accurate answers

The key enhancement is the ability to ground responses in specific knowledge rather 

In [27]:
# RagLLM - улучшенная версия Агента с памятью + поддержкой RAG
# RAG позволяет:
# 1. Искать релевантную информацию в базе знаний
# 2. Использовать найденную информацию в ответах
# 3. Давать более точные и информированные ответы
class RagLLM(MemoryLLM):
    """LLM with conversation memory and RAG capability."""
    
    def __init__(self, 
                 openai_key: str, 
                 pinecone_key: str, 
                 index_name: str, 
                 model: str = "gpt-4o-mini"):
        # Инициализируем:
        # 1. Базовый класс (память диалога)
        # 2. Компоненты для RAG (поиск информации)
        super().__init__(openai_key, model)
        self.pinecone_key = pinecone_key
        self.index_name = index_name
        self.embedding_generator = EmbeddingGenerator(openai_key)
        self.vector_store = VectorStore(pinecone_key, index_name)
        
    def _get_relevant_chunks(self, text: str, top_k: int = 3) -> List[str]:
        """Get relevant text chunks using vector search."""
        try:
            # Процесс поиска:
            # 1. Создаем эмбеддинг для запроса
            # Get query embedding
            query_vector = self.embedding_generator.create_embedding(text)
            
            # Query vector store
            # 2. Ищем похожие вектора в базе
            matches = self.vector_store.query_vectors(
                query_vector=query_vector,
                top_k=top_k
            )
            
            # Extract texts from metadata
            # 3. Извлекаем тексты из метаданных
            return [match['metadata'].get('text', '') for match in matches]
            
        except Exception as e:
            print(f"Error getting relevant chunks: {e}")
            return []
            
    def process_message(self, message: str) -> str:
        """Process message with conversation history and relevant context."""
        try:
            # Get relevant context
            # 1. Ищем релевантную информацию
            context_chunks = self._get_relevant_chunks(message)
            # Объединяем найденные (3) части в один текст
            context = "\n".join(context_chunks)
            print(f"\nFound {len(context_chunks)} relevant chunks")
            if context_chunks:
                print(f"Sample context (first 50 chars): {context_chunks[0][:50]}...")
            
            # Prepare messages
            # 2. Готовим сообщения для API:
            # - Задаем Системное сообщение (промпт) с контекстом (инфа из RAG)
            # - История диалога (память)
            # - Текущее сообщение пользователя
            messages = [{
                "role": "system",
                "content": f"""You are a helpful marketing assistant. Answer just if you sure about correct info 100%. If not say you not sure. 
                Use this context when relevant:
                {context}
                """
            }]
            messages.extend([msg.dict() for msg in self.history])
            messages.append({"role": "user", "content": message})

            # 3. Делаем запрос к API
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model,
                    "messages": messages
                }
            )
            response.raise_for_status()
            
            # Get response and update history
            # 4. Обрабатываем ответ:
            # - Получаем текст ответа
            assistant_message = response.json()['choices'][0]['message']['content']
            # - Сохраняем в историю диалога
            self.history.append(Message(role="user", content=message))
            self.history.append(Message(role="assistant", content=assistant_message))
            
            return assistant_message
            
        except Exception as e:
            return f"Error: {str(e)}"

In [26]:
print("\n=== Testing Document Processing ===")
with open("Global Sourcing RFI Workflow.txt", "r") as f:
    text = f.read()
    success = process_and_store_document(
        text=text,
        document_id="sourcing_workflow",
        openai_key=os.environ["OPENAI_API_KEY"],
        pinecone_key=os.environ["PINECONE_API_KEY"],
        index_name="augmentedllm"
    )
    print(f"\nDocument processing {'succeeded' if success else 'failed'}")


=== Testing Document Processing ===

Created 13 chunks
Sample chunk (first 50 chars): ﻿Global Sourcing RFI Workflow


Global Sourcing RF...
Generated embedding, sample of first 3 dimensions: [-0.009754701, 0.023795739, 0.010035283]
Generated embedding, sample of first 3 dimensions: [-0.04238617, 0.009184542, 0.010165098]
Generated embedding, sample of first 3 dimensions: [-0.03158922, 0.03678677, 0.020493945]
Generated embedding, sample of first 3 dimensions: [-0.0501792, 0.040126923, 0.057081576]
Generated embedding, sample of first 3 dimensions: [-0.04344335, 0.011441238, 0.02943367]
Generated embedding, sample of first 3 dimensions: [-0.021920724, 0.040596727, 0.025479855]
Generated embedding, sample of first 3 dimensions: [-0.047885858, -0.004947266, 0.02059778]
Generated embedding, sample of first 3 dimensions: [-0.054924604, -0.008015131, 0.035972822]
Generated embedding, sample of first 3 dimensions: [-0.047217026, -0.02386938, 0.020438973]
Generated embedding, sample of first 

In [28]:
# Test 3b: LLM with Memory and RAG
print("\n=== Testing LLM with Memory and RAG ===")
rag_llm = RagLLM(os.environ["OPENAI_API_KEY"], os.environ["PINECONE_API_KEY"], "augmentedllm")
rag_test = DialogueTest(rag_llm)
rag_test.chat("Hi! My name is Viacheslav.")
rag_test.chat("Do you remember my name?")
rag_test.chat("What you khow about The Global Sourcing RFI Company?")
rag_test.chat("We spent $10,000 on a marketing campaign that generated $50,000 in revenue with a 10% margin. What was our ROMI?")


=== Testing LLM with Memory and RAG ===

User: Hi! My name is Viacheslav.
Generated embedding, sample of first 3 dimensions: [0.015347642, -0.020478658, -0.042047087]

Connected to Pinecone host: augmentedllm-cxx1dj3.svc.aped-4627-b74a.pinecone.io

Querying Pinecone for top 3 matches

Found 3 relevant chunks
Sample context (first 50 chars): izing in supplier identification and matching. Vie...
Assistant: Hello, Viacheslav! How can I assist you today?

User: Do you remember my name?
Generated embedding, sample of first 3 dimensions: [0.036457118, -0.07861758, -0.049314216]

Querying Pinecone for top 3 matches

Found 3 relevant chunks
Sample context (first 50 chars): e workflow will check for available contact inform...
Assistant: Yes, your name is Viacheslav. How can I assist you further?

User: What you khow about The Global Sourcing RFI Company?
Generated embedding, sample of first 3 dimensions: [0.0016378722, -0.012900318, 0.034605835]

Querying Pinecone for top 3 matches

Found 3 r

'To calculate the Return on Marketing Investment (ROMI), you can use the following formula:\n\n\\[ \\text{ROMI} = \\frac{\\text{Revenue} - \\text{Marketing Investment}}{\\text{Marketing Investment}} \\]\n\nIn this case:\n\n- Revenue = $50,000\n- Marketing Investment = $10,000\n\nFirst, calculate the profit generated:\n\n\\[ \\text{Profit} = \\text{Revenue} \\times \\text{Margin} = 50,000 \\times 0.10 = 5,000 \\]\n\nNow, plug in the values into the ROMI formula:\n\n\\[ \\text{ROMI} = \\frac{50,000 - 10,000}{10,000} = \\frac{40,000}{10,000} = 4 \\]\n\nSo, the ROMI is 4, or 400%. This means for every dollar spent on marketing, you generated 4 dollars in revenue.'

## Part 4: Adding Specialized Tools

In our final enhancement, we add specialized tools to our agent. This example implements marketing analytics tools, specifically ROMI calculation.

Key Features:
- Tool definition system
- Automatic tool selection by the LLM
- Result integration into responses

This demonstrates how we can extend our agent with specific computational capabiliti

In [30]:
# MarketingTools - класс с инструментами для маркетинговой аналитики
# Здесь мы определяем инструменты, которые может использовать Агент
class MarketingTools:
    """Provides marketing analytics calculations."""
    
    @staticmethod
    def get_tool_definitions() -> List[Dict]:
        # Определяем инструменты в формате, который понимает OpenAI API
        # Для каждого инструмента указываем:
        # 1. Название и описание
        # 2. Входные параметры
        # 3. Правила использования
        return [{
            "type": "function",
            "function": {
                "name": "calculate_romi",
                "description": "Calculate Return on Marketing Investment (ROMI)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "revenue": {
                            "type": "number",
                            "description": "Total revenue generated from marketing campaign"
                        },
                        "marketing_cost": {
                            "type": "number",
                            "description": "Total cost of marketing campaign"
                        },
                        "margin_percent": {
                            "type": "number",
                            "description": "Profit margin percentage (0-100)"
                        }
                    },
                    "required": ["revenue", "marketing_cost", "margin_percent"]
                }
            }
        }]
    
    @staticmethod
    def calculate_romi(revenue: float, marketing_cost: float, margin_percent: float) -> float:
        """
        Calculate ROMI using the formula: ((Revenue * Margin%) - Marketing Cost) / Marketing Cost * 100
        Returns percentage value
        """
        # Проверяем входные данные:
        if marketing_cost <= 0:
            raise ValueError("Marketing cost must be greater than zero")
        if not (0 <= margin_percent <= 100):
            raise ValueError("Margin percentage must be between 0 and 100")

        # Рассчитываем ROMI:
        # 1. Переводим процент в множитель (10% -> 0.1)
        margin_multiplier = margin_percent / 100
        # 2. Считаем прибыль (выручка * маржа - затраты)
        profit = revenue * margin_multiplier - marketing_cost
        # 3. Считаем ROMI ((прибыль / затраты) * 100%)
        romi = (profit / marketing_cost) * 100
        # Округляем до 2 знаков после запятой
        return round(romi, 2)

In [31]:
# ToolEnabledLLM - финальная версия нашего бота
# Он объединяет все возможности:
# 1. Память (от MemoryLLM)
# 2. Поиск контекста (от RagLLM)
# 3. Использование инструментов
class ToolEnabledLLM(RagLLM):
    """LLM with memory, RAG and tool usage capability."""
    
    def __init__(self, 
                 openai_key: str, 
                 pinecone_key: str, 
                 index_name: str, 
                 model: str = "gpt-4o-mini"):
        # При создании:
        # 1. Инициализируем родительский класс (RagLLM)
        super().__init__(openai_key, pinecone_key, index_name, model)
        # 2. Добавляем инструменты
        self.tools = MarketingTools()
    
    def process_message(self, message: str) -> str:
        """Process message with tools, memory and RAG."""
        try:
            # 1. Поиск релевантного контекста:
            # Get relevant context
            # - Ищем релевантный контекст через RAG
            context_chunks = self._get_relevant_chunks(message)
            # - Выводим статистику для отладки
            context = "\n".join(context_chunks)
            # - Выводим статистику для отладки
            print(f"\nFound {len(context_chunks)} relevant chunks")
            if context_chunks:
                print(f"Sample context (first 50 chars): {context_chunks[0][:50]}...")
            
            # Prepare messages
            # 2. Готовим сообщения для API:
            # - Создаем системный промпт с контекстом и Добавляем инструкции по использованию инструментов
            messages = [{
                "role": "system",
                "content": f"""You are a marketing analytics assistant. 
                Use this context when relevant:
                {context}
                
                When asked about marketing metrics, use the calculate_romi tool.
                ROMI (Return on Marketing Investment) shows the profitability of marketing spending.
                """
            }]
            # Добавляем историю диалога
            messages.extend([msg.dict() for msg in self.history])
            # Добавляем текущее сообщение
            messages.append({"role": "user", "content": message})
            
            # Make API request with tools
            # 3. Делаем первый запрос к API с инструментами
            # - Отправляем сообщения и описания инструментов
            # - LLM может решить использовать инструменты
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model,
                    "messages": messages,
                    "tools": self.tools.get_tool_definitions() # tools (!)
                }
            )
            response.raise_for_status() 
            
            # Process response and tool calls
            # 5. Обработка ответа:
            # - Получаем JSON с ответом
            # - Извлекаем сообщение ассистента
            response_data = response.json()
            assistant_message = response_data['choices'][0]['message']
            # 6. Проверяем использование инструментов:
            # Если Агент решил использовать инструмент (посчитать ROMI):
            if tool_calls := assistant_message.get('tool_calls'):
                # - Выводим сообщение о начале обработки
                # - Вызываем инструменты и получаем результаты
                print("\nProcessing tool calls...")
                tool_results = self._handle_tool_calls(tool_calls)
                # Add tool results to conversation
                # 8. Добавляем результаты в контекст:
                # - Сообщение ассистента с вызовом инструмента и Результаты работы инструмента (значение ROMI)
                messages.extend([
                    {
                        "role": "assistant",
                        "content": assistant_message.get('content'),
                        "tool_calls": tool_calls
                    },
                    {
                        "role": "tool",
                        "content": json.dumps(tool_results),
                        "tool_call_id": tool_calls[0]['id']
                    }
                ])
                
                # Get final response with tool results
                # 9. Второй запрос к API:
                # - Отправляем обновленный контекст с результатами
                # - LLM формирует финальный ответ с учетом результатов
                final_response = requests.post(
                    "https://api.openai.com/v1/chat/completions",
                    headers={
                        "Authorization": f"Bearer {self.api_key}",
                        "Content-Type": "application/json"
                    },
                    json={
                        "model": self.model,
                        "messages": messages
                    }
                )
                final_response.raise_for_status()
                assistant_message = final_response.json()['choices'][0]['message']
            
            # Update conversation history
            # 10. Обновление истории:
            # - Сохраняем сообщение пользователя
            # - Сохраняем финальный ответ ассистента
            self.history.append(Message(role="user", content=message))
            self.history.append(Message(role="assistant", content=assistant_message['content']))

            # 11. Возвращаем Финальный ответ:
            return assistant_message['content']
            
        except Exception as e:
            return f"Error: {str(e)}"
            
    def _handle_tool_calls(self, tool_calls: List[Dict]) -> Dict:
        """Process tool calls and return results."""
        # Обрабатываем каждый вызов инструмента:
        # Для каждого вызова:
        # 1. Проверяем, какой инструмент вызван
        # 2. Получаем параметры
        # 3. Выполняем расчет
        results = {}
        
        for call in tool_calls:
            # [VIDEO] Если вызван инструмент для расчета ROMI:
            if call['function']['name'] == 'calculate_romi':
                try:
                    # Извлекаем параметры из JSON
                    args = json.loads(call['function']['arguments'])
                    # Вызываем функцию расчета
                    result = self.tools.calculate_romi(
                        revenue=args['revenue'],
                        marketing_cost=args['marketing_cost'],
                        margin_percent=args['margin_percent']
                    )
                    print(f"Calculated ROMI: {result}%")
                    # Сохраняем результат
                    results[call['id']] = result
                except Exception as e:
                    # Если произошла ошибка, сохраняем её
                    results[call['id']] = f"Error calculating ROMI: {str(e)}"
                    
        return results

In [33]:
# Test 4: LLM with Tools
print("\n=== Testing Tool-enabled LLM ===")
tool_llm = ToolEnabledLLM(os.environ["OPENAI_API_KEY"], os.environ["PINECONE_API_KEY"], "augmentedllm")
tool_test = DialogueTest(tool_llm)
tool_test.chat("Hi! My name is Slava?")
tool_test.chat("Do you remember my name?")
tool_test.chat("How will we translate Vietnamese Data into Vietnamese?")
tool_test.chat("We spent $10,000 on a marketing campaign that generated $50,000 in revenue with a 10% margin. What was our ROMI?")
tool_test.chat("How about a campaign where we spent $200,000 and got $4,000,000 in revenue with 50% margin?")


=== Testing Tool-enabled LLM ===

User: Hi! My name is Slava?
Generated embedding, sample of first 3 dimensions: [0.008466213, -0.008110435, -0.04214]

Connected to Pinecone host: augmentedllm-cxx1dj3.svc.aped-4627-b74a.pinecone.io

Querying Pinecone for top 3 matches

Found 3 relevant chunks
Sample context (first 100 chars): izing in supplier identification and matching. Vientam Direct Sourcing is designed to connect medium...
Assistant: Hello Slava! How can I assist you today?

User: Do you remember my name?
Generated embedding, sample of first 3 dimensions: [0.036457118, -0.07861758, -0.049314216]

Querying Pinecone for top 3 matches

Found 3 relevant chunks
Sample context (first 100 chars): e workflow will check for available contact information: if present, an email will be sent to the cl...
Assistant: Yes, I remember your name, Slava! How can I help you today?

User: How will we translate Vietnamese Data into Vietnamese?
Generated embedding, sample of first 3 dimensions: [-0.000

'For the campaign where you spent $200,000 and generated $4,000,000 in revenue with a 50% margin, your ROMI is 900%. This indicates a highly profitable marketing investment. If you need further analysis or assistance, feel free to ask!'